In [0]:
entity_name = dbutils.widgets.get("entity_name").capitalize()

In [0]:
if entity_name.endswith('incremental'):
    load_type = 'incremental'
else:
    load_type = 'full'

In [0]:
%run ./CommonUtils

In [0]:
%run ./CreateOrReplaceTables

In [0]:
%run ./SCD_Strategy

In [0]:
src_df = spark.read.parquet(f"{get_mnt('landing')}/{entity_name}/{entity_name}.parquet")
src_df = src_df.withColumn("processing_dttm",current_timestamp())
if load_type == 'full':
    crt_execute(entity_name,'bronze',truncate_flag=False,src_df=src_df)
else:
    pk = pk_map_bronze(entity_name.split('_inc')[0].lower())
    SCD1(entity_name,pk=pk,ingestion_layer="bronze").execute_scd_1()

In [0]:
print("Bronze Ingestion Completed")